In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
from response.feature import Feature

In [3]:
path = "../../corpus/hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [4]:
def make_Xy(convs):
    X = []
    y = []
    for conv in convs:
        for i, ut in enumerate( conv ):
            if ut.is_utt_level_error():
                continue
            X.append(ut.utt)
            if ut.is_exist_type():
                y.append(1)
            else:
                y.append(0)
    return X, y

In [5]:
X, y = make_Xy(convs)

In [6]:
len(y)

3814

In [7]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5, stratify=y)

In [8]:
F = Feature()
F_path = "../X_y_data/response2/"
F_name = "Classify_F2.pickle"
featureM = DataManager(F_path)
F.make_features(X_train_str)
featureM.save_data(F_name, F)

KeyboardInterrupt: 

In [ ]:
print(F.feature_num)
X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

56198


In [ ]:
from sklearn import svm
# clf = AdaBoostClassifier()
clf = svm.SVC(kernel='rbf', gamma =0.0001, C=1000)

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[817  38]
 [ 27 263]]
accuracy =  0.9432314410480349
precision =  0.8737541528239202
recall =  0.906896551724138
f1 score =  0.8900169204737732


In [ ]:
model_path = "../models/response2/"
model_name = "Classify_M2.pickle"
modelM = DataManager(model_path)
modelM.save_data(model_name, clf)

success save : ../models/response2/Classify_M2.pickle
